# Generate Payment and Value of Care dataframe

In [1]:
import pandas as pd
import numpy as np
import warnings
from IPython.utils import io
import sys
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

main_dir = '/Users/kenlocey/Desktop/Rush/CMS_HospitalArchives/'

In [2]:
df = pd.read_pickle(main_dir + 'Payment_and_ValueOfCare/CombinedFiles_Payment_and_ValueOfCare/' + 'Facility.pkl')
print('Payment and Value of Care df:', df.shape)

df.drop(labels=['Address', 'City', 'County Name', 'Facility Name', 
                    'Measure End Date', 'Measure Start Date', 'Payment Footnote',
                    'Phone Number', 'State', 'Value of Care Footnote', 'ZIP Code',
                   ], axis=1, inplace=True)

df = df.filter(items=['Facility ID', 
                        'file_month',
                        'file_year',
                        'Denominator', 
                        'Higher Estimate',
                        'Lower Estimate',
                        'Payment',
                        'Payment Category',
                        'Payment Measure ID',
                        'Payment Measure Name',
                        'Value of Care Category',
                        'Value of Care Display ID',
                        #'Value of Care Display Name',
                        ], axis=1)



df['Value of Care Display ID'].fillna('Not Available', inplace=True)
df['Value of Care Category'].fillna('Not Available', inplace=True)
df.head()

Payment and Value of Care df: (569101, 24)


,Facility ID,file_month,file_year,Denominator,Higher Estimate,Lower Estimate,Payment,Payment Category,Payment Measure ID,Payment Measure Name,Value of Care Category,Value of Care Display ID
0,010001,01,2023,311,"$29,027","$24,890","$26,894",No Different Than the National Average Payment,PAYM_30_AMI,Payment for heart attack patients,Average Mortality and Average Payment,MORT_PAYM_30_AMI
1,010001,01,2023,620,"$18,783","$16,947","$17,835",No Different Than the National Average Payment,PAYM_30_HF,Payment for heart failure patients,Better Mortality and Average Payment,MORT_PAYM_30_HF
2,010001,01,2023,397,"$20,653","$19,430","$20,182",No Different Than the National Average Payment,PAYM_30_PN,Payment for pneumonia patients,Average Mortality and Average Payment,MORT_PAYM_30_PN
3,010001,01,2023,101,"$23,309","$20,104","$21,623",No Different Than the National Average Payment,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,Average Complications and Average Payment,COMP_PAYM_90_HIP_KNEE
4,010005,01,2023,48,"$29,475","$22,668","$25,814",No Different Than the National Average Payment,PAYM_30_AMI,Payment for heart attack patients,Average Mortality and Average Payment,MORT_PAYM_30_AMI


In [3]:
print(len(df['Payment Measure Name'].unique()))
print(df['Payment Measure Name'].unique())
print('\n')

print(len(df['Value of Care Display ID'].unique()))
print(df['Value of Care Display ID'].unique())
print('\n')

#print(len(pvc_df['Value of Care Category'].unique()))
#print(pvc_df['Value of Care Category'].unique())
#print('\n')

df['Measure'] = df['Value of Care Display ID'] + ' — ' + df['Payment Measure Name']
df.drop(labels=['Payment Measure Name', 'Value of Care Display ID'], axis=1, inplace=True)

4
['Payment for heart attack patients' 'Payment for heart failure patients'
 'Payment for pneumonia patients'
 'Payment for hip/knee replacement patients']


5
['MORT_PAYM_30_AMI' 'MORT_PAYM_30_HF' 'MORT_PAYM_30_PN'
 'COMP_PAYM_90_HIP_KNEE' 'Not Available']




In [4]:
    
d = {
    'Not Available — Payment for heart attack patients': 'MORT_PAYM_30_AMI — Payment for heart attack patients',
    'Not Available — Payment for heart failure patients': 'MORT_PAYM_30_HF — Payment for heart failure patients',
    'Not Available — Payment for pneumonia patients': 'MORT_PAYM_30_PN — Payment for pneumonia patients',    
    }  

df['Measure'].replace(to_replace=d, inplace=True)

measures = sorted(df['Measure'].unique())
for m in measures:
    print(m)


COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients
MORT_PAYM_30_AMI — Payment for heart attack patients
MORT_PAYM_30_HF — Payment for heart failure patients
MORT_PAYM_30_PN — Payment for pneumonia patients


In [5]:
measure_ids = sorted(df['Measure'].unique())
main_df = pd.DataFrame(columns=['Facility ID', 'file_month', 'file_year'])

for i, m_id in enumerate(measure_ids):
    
    tdf = df[df['Measure'] == m_id]
    measures = sorted(tdf['Measure'].unique())
    
    df2 = pd.DataFrame(columns=['Facility ID', 'file_month', 'file_year'])

    for j, m in enumerate(measures):
    
        tdf2 = tdf[tdf['Measure'] == m]
        for n in list(tdf2):
            if n == 'Measure' or n in ['Facility ID', 'file_month', 'file_year']:
                continue

            else:
                
                if n == 'Denominator':                
                    tdf2[n] = pd.to_numeric(tdf2[n], errors='coerce')
                
                elif n in ['Higher Estimate', 'Lower Estimate', 'Payment']:
                    tdf2[n] = tdf2[n].replace("[$,]", "", regex=True)
                    tdf2[n] = pd.to_numeric(tdf2[n], errors='coerce')
                
                tdf2.rename(columns={n: m + ' (' + n + ')'}, inplace=True)
        
        tdf2.drop(labels=['Measure'], axis=1, inplace=True)
        
        df2 = df2.merge(tdf2, on=['Facility ID', 'file_month', 
                                          'file_year'], how='outer')
    
    main_df = main_df.merge(df2, on=['Facility ID', 'file_month', 
                                          'file_year'], how='outer')


df = main_df.copy(deep=True)
del main_df, df2
print(df.shape)
df.dropna(how='all', axis=1, inplace=True)
print(df.shape)
df.head()


(151810, 31)
(151810, 31)


,Facility ID,file_month,file_year,COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Denominator),COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Higher Estimate),COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Lower Estimate),COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Payment),COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Payment Category),COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Payment Measure ID),COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Value of Care Category),MORT_PAYM_30_AMI — Payment for heart attack patients (Denominator),MORT_PAYM_30_AMI — Payment for heart attack patients (Higher Estimate),MORT_PAYM_30_AMI — Payment for heart attack patients (Lower Estimate),MORT_PAYM_30_AMI — Payment for heart attack patients (Payment),MORT_PAYM_30_AMI — Payment for heart attack patients (Payment Category),MORT_PAYM_30_AMI — Payment for heart attack patients (Payment Measure ID),MORT_PAYM_30_AMI — Payment for heart attack patients (Value of Care Category),MORT_PAYM_30_HF — Payment for heart failure patients (Denominator),MORT_PAYM_30_HF — Payment for heart failure patients (Higher Estimate),MORT_PAYM_30_HF — Payment for heart failure patients (Lower Estimate),MORT_PAYM_30_HF — Payment for heart failure patients (Payment),MORT_PAYM_30_HF — Payment for heart failure patients (Payment Category),MORT_PAYM_30_HF — Payment for heart failure patients (Payment Measure ID),MORT_PAYM_30_HF — Payment for heart failure patients (Value of Care Category),MORT_PAYM_30_PN — Payment for pneumonia patients (Denominator),MORT_PAYM_30_PN — Payment for pneumonia patients (Higher Estimate),MORT_PAYM_30_PN — Payment for pneumonia patients (Lower Estimate),MORT_PAYM_30_PN — Payment for pneumonia patients (Payment),MORT_PAYM_30_PN — Payment for pneumonia patients (Payment Category),MORT_PAYM_30_PN — Payment for pneumonia patients (Payment Measure ID),MORT_PAYM_30_PN — Payment for pneumonia patients (Value of Care Category)
0,010001,01,2023,101.0,23309.0,20104.0,21623.0,No Different Than the National Average Payment,PAYM_90_HIP_KNEE,Average Complications and Average Payment,311.0,29027.0,24890.0,26894.0,No Different Than the National Average Payment,PAYM_30_AMI,Average Mortality and Average Payment,620.0,18783.0,16947.0,17835.0,No Different Than the National Average Payment,PAYM_30_HF,Better Mortality and Average Payment,397.0,20653.0,19430.0,20182.0,No Different Than the National Average Payment,PAYM_30_PN,Average Mortality and Average Payment
1,010005,01,2023,133.0,20281.0,17873.0,19040.0,Less Than the National Average Payment,PAYM_90_HIP_KNEE,Average Complications and Lower Payment,48.0,29475.0,22668.0,25814.0,No Different Than the National Average Payment,PAYM_30_AMI,Average Mortality and Average Payment,142.0,19040.0,15749.0,17287.0,No Different Than the National Average Payment,PAYM_30_HF,Worse Mortality and Average Payment,343.0,19563.0,18407.0,18639.0,No Different Than the National Average Payment,PAYM_30_PN,Worse Mortality and Average Payment
2,010006,01,2023,232.0,23148.0,20905.0,21994.0,Greater Than the National Average Payment,PAYM_90_HIP_KNEE,Average Complications and Higher Payment,287.0,30977.0,26395.0,28589.0,No Different Than the National Average Payment,PAYM_30_AMI,Worse Mortality and Average Payment,451.0,19918.0,17693.0,18745.0,No Different Than the National Average Payment,PAYM_30_HF,Average Mortality and Average Payment,511.0,20779.0,19452.0,20218.0,No Different Than the National Average Payment,PAYM_30_PN,Average Mortality and Average Payment
3,010007,01,2023,NaN,NaN,NaN,NaN,Number of Cases Too Small,PAYM_90_HIP_KNEE,Not Available,NaN,NaN,NaN,NaN,Number of Cases Too Small,PAYM_30_AMI,Not Available,43.0,19839.0,15128.0,17397.0,No Different Than the National Average Payment,PAYM_30_HF,Average Mortality and Average Payment,101.0,19791.0,18753.0,19222.0,No Different Than the National Average

In [6]:
for i, n in enumerate(list(df)):
    if n in ['Facility ID', 'file_month', 'file_year']:
        continue
            
    df.rename(columns={n: 'Payment and Value of Care: ' + n}, inplace=True)
    
print(df.shape)
df.head()

(151810, 31)


,Facility ID,file_month,file_year,Payment and Value of Care: COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Denominator),Payment and Value of Care: COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Higher Estimate),Payment and Value of Care: COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Lower Estimate),Payment and Value of Care: COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Payment),Payment and Value of Care: COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Payment Category),Payment and Value of Care: COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Payment Measure ID),Payment and Value of Care: COMP_PAYM_90_HIP_KNEE — Payment for hip/knee replacement patients (Value of Care Category),Payment and Value of Care: MORT_PAYM_30_AMI — Payment for heart attack patients (Denominator),Payment and Value of Care: MORT_PAYM_30_AMI — Payment for heart attack patients (Higher Estimate),Payment and Value of Care: MORT_PAYM_30_AMI — Payment for heart attack patients (Lower Estimate),Payment and Value of Care: MORT_PAYM_30_AMI — Payment for heart attack patients (Payment),Payment and Value of Care: MORT_PAYM_30_AMI — Payment for heart attack patients (Payment Category),Payment and Value of Care: MORT_PAYM_30_AMI — Payment for heart attack patients (Payment Measure ID),Payment and Value of Care: MORT_PAYM_30_AMI — Payment for heart attack patients (Value of Care Category),Payment and Value of Care: MORT_PAYM_30_HF — Payment for heart failure patients (Denominator),Payment and Value of Care: MORT_PAYM_30_HF — Payment for heart failure patients (Higher Estimate),Payment and Value of Care: MORT_PAYM_30_HF — Payment for heart failure patients (Lower Estimate),Payment and Value of Care: MORT_PAYM_30_HF — Payment for heart failure patients (Payment),Payment and Value of Care: MORT_PAYM_30_HF — Payment for heart failure patients (Payment Category),Payment and Value of Care: MORT_PAYM_30_HF — Payment for heart failure patients (Payment Measure ID),Payment and Value of Care: MORT_PAYM_30_HF — Payment for heart failure patients (Value of Care Category),Payment and Value of Care: MORT_PAYM_30_PN — Payment for pneumonia patients (Denominator),Payment and Value of Care: MORT_PAYM_30_PN — Payment for pneumonia patients (Higher Estimate),Payment and Value of Care: MORT_PAYM_30_PN — Payment for pneumonia patients (Lower Estimate),Payment and Value of Care: MORT_PAYM_30_PN — Payment for pneumonia patients (Payment),Payment and Value of Care: MORT_PAYM_30_PN — Payment for pneumonia patients (Payment Category),Payment and Value of Care: MORT_PAYM_30_PN — Payment for pneumonia patients (Payment Measure ID),Payment and Value of Care: MORT_PAYM_30_PN — Payment for pneumonia patients (Value of Care Category)
0,010001,01,2023,101.0,23309.0,20104.0,21623.0,No Different Than the National Average Payment,PAYM_90_HIP_KNEE,Average Complications and Average Payment,311.0,29027.0,24890.0,26894.0,No Different Than the National Average Payment,PAYM_30_AMI,Average Mortality and Average Payment,620.0,18783.0,16947.0,17835.0,No Different Than the National Average Payment,PAYM_30_HF,Better Mortality and Average Payment,397.0,20653.0,19430.0,20182.0,No Different Than the National Average Payment,PAYM_30_PN,Average Mortality and Average Payment
1,010005,01,2023,133.0,20281.0,17873.0,19040.0,Less Than the National Average Payment,PAYM_90_HIP_KNEE,Average Complications and Lower Payment,48.0,29475.0,22668.0,25814.0,No Different Than the National Average Payment,PAYM_30_AMI,Average Mortality and Average Payment,142.0,19040.0,15749.0,17287.0,No Different Than the National Average Payment,PAYM_30_HF,Worse Mortality and Average Payment,343.0,19563.0,18407.0,18639.0,No Different Than the National Average Payment,PAYM_30_PN,Worse Mortality and Average Payment
2,010006,01,2023,232.0,23148.0,20905.0,21994.0,Greater Than the National Average Payment,PAYM_90_HIP_KNEE,Average Compl

In [7]:
df.to_pickle('dataframe_data/Payment_and_ValueOfCare_df.pkl', protocol=5)